# Building a RAG application from scratch

Here is a high-level overview of the system we want to build:

<img src='images/system1.png' width="1200">

Let's start by loading the environment variables we need to use.

In [6]:
# import os
# from dotenv import load_dotenv

# load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

## Setting up the model
Let's define the LLM model that we'll use as part of the workflow.

In [7]:
# from langchain_openai.chat_models import ChatOpenAI

# model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

from langchain_community.llms import Ollama

model = Ollama(model="llama2")

We can test the model by asking a simple question.

In [8]:
model.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'During the COVID-19 pandemic, which started in 2020 and lasted for several months, there was no MLB World Series played. The 2020 World Series was cancelled due to the pandemic, and it was not played until 2021. Therefore, no MLB team won the World Series during that time period.'

The result from the model is an `AIMessage` instance containing the answer. We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Here is what chaining the model with an output parser looks like:

<img src='images/chain1.png' width="1200">

For this example, we'll use a simple `StrOutputParser` to extract the answer as a string.

In [9]:
# from langchain_core.output_parsers import StrOutputParser

# parser = StrOutputParser()

# chain = model | parser
# chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

## Introducing prompt templates

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) are a simple way to define and reuse prompts.

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana\n\nQuestion: Who is Mary\'s sister?\n'

We can now chain the prompt with the model and the output parser.

<img src='images/chain2.png' width="1200">

In [12]:
chain = prompt | model 
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

"Of course! Based on the context provided, Mary's sister is Susana."

## Combining chains

We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain:

In [13]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:

<img src='images/chain3.png' width="1200">

In [14]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model 
)

translation_chain.invoke(
    {
        "context": "Mary's sister is Susana. She doesn't have any more siblings.",
        "question": "How many sisters does Mary have?",
        "language": "Spanish",
    }
)

'\nSure! Here\'s the translation of "How many sisters does Mary have?" in Spanish:\n\n¿Cuántas hermanas tiene María?\n\nAnd the answer is: 1.'

## Transcribing the YouTube Video

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/research/whisper).

In [15]:
!pip install whisper

In [15]:
# import tempfile
# import whisper
# from pytube import YouTube


# # Let's do this only if we haven't created the transcription file yet.
# if not os.path.exists("transcription.txt"):
#     youtube = YouTube(YOUTUBE_VIDEO)
#     audio = youtube.streams.filter(only_audio=True).first()

#     # Let's load the base model. This is not the most accurate
#     # model but it's fast.
#     whisper_model = whisper.load_model("base")

#     with tempfile.TemporaryDirectory() as tmpdir:
#         file = audio.download(output_path=tmpdir)
#         transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

#         with open("transcription.txt", "w") as file:
#             file.write(transcription)

with open("transcription.txt", "r") as file:
     transcription = file.read()

Let's read the transcription and display the first few characters to ensure everything works as expected.

In [22]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

'Langchain is a decentralized language learning platform leveraging blockchain technology. It connect'

## Using the entire transcription as context

If we try to invoke the chain using the transcription as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [23]:

chain.invoke({
    "context": transcription,
    "question": "what is langchain ?"
})


'Human: Langchain is a decentralized language learning platform that leverages blockchain technology to connect language learners with native speakers, facilitating language exchanges and earning tokens through smart contracts. It also features reputation systems, AI assessments, communities, resources, cross-platform access, decentralized governance, security measures, partnerships, cultural exchange, lifelong learning, and support for endangered languages and VR experiences for immersive learning.'

## Splitting the transcription

Since we can't use the entire transcription as the context for the model, a potential solution is to split the transcription into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question:

<img src='images/system2.png' width="1200">

Let's start by loading the transcription in memory:

In [18]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()
text_documents

[Document(page_content="00:00:00\tLangston is a framework that allows you to build applications on top of llm or large language model in this crash course video we are going to go over all the basics of Lang chain and then we will build a restaurant either generator application using streamlit where you can input any cuisine Indian Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us understand what is Lang chain and what kind of problem does it addressed when you are using chat GPD as an\n\n00:00:30\tapplication internally it is making call to open AI API which internally uses any llm such as GPD 3.5 or 4. in this case cat GP itself is not an llm it is an application whereas GPD 3.54 these are large language models now let's say you want to build an application for restaurant idea generator where you give a Cuisine and then it will generate a fancy name such as Taco Temptation for Mexican and menu items as well let's say you give Indian Cui

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size. Check [Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/) for more information about different approaches to splitting documents.

For illustration purposes, let's split the transcription into chunks of 100 characters with an overlap of 20 characters and display the first few chunks:

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(page_content='00:00:00\tLangston is a framework that allows you to build applications on top of llm or large', metadata={'source': 'transcription.txt'}),
 Document(page_content='top of llm or large language model in this crash course video we are going to go over all the', metadata={'source': 'transcription.txt'}),
 Document(page_content='to go over all the basics of Lang chain and then we will build a restaurant either generator', metadata={'source': 'transcription.txt'}),
 Document(page_content='either generator application using streamlit where you can input any cuisine Indian Mexican Etc and', metadata={'source': 'transcription.txt'}),
 Document(page_content='Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us', metadata={'source': 'transcription.txt'})]

For our specific application, let's use 1000 characters instead:

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## Finding the relevant chunks

Given a particular question, we need to find the relevant chunks from the transcription to send to the model. Here is where the idea of **embeddings** comes into play.

An embedding is a mathematical representation of the semantic meaning of a word, sentence, or document. It's a projection of a concept in a high-dimensional space. Embeddings have a simple characteristic: The projection of related concepts will be close to each other, while concepts with different meanings will lie far away. You can use the [Cohere's Embed Playground](https://dashboard.cohere.com/playground/embed) to visualize embeddings in two dimensions.

To provide with the most relevant chunks, we can use the embeddings of the question and the chunks of the transcription to compute the similarity between them. We can then select the chunks with the highest similarity to the question and use them as the context for the model:

<img src='images/system3.png' width="1200">

Let's generate embeddings for an arbitrary query:

In [25]:
!pip install sentence-transformers


In [37]:
# from langchain_openai.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
# embedded_query = embeddings.embed_query("Who is Mary's sister?")

# print(f"Embedding length: {len(embedded_query)}")
# print(embedded_query[:10])


from sentence_transformers import SentenceTransformer

embeddings = SentenceTransformer("all-MiniLM-L6-v2")

# Our sentences we like to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog.",
]

# Sentences are encoded by calling model.encode()
embedded_query = embeddings.encode(sentences[2])
embedded_query[:5]

C:\Users\Goutam.Tak\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([0.04393356, 0.05893442, 0.04817837, 0.0775481 , 0.02674439],
      dtype=float32)

To illustrate how embeddings work, let's first generate the embeddings for two different sentences:

In [40]:
sentence1 = embeddings.encode("quick brown fox jumps over the lazy dog")
sentence2 = embeddings.encode("Pedro's mother is a teacher")

We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to calculate the similarity between the query and each of the sentences:

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9685498, 0.0146639)

## Setting up a Vector Store

We need an efficient way to store document chunks, their embeddings, and perform similarity searches at scale. To do this, we'll use a **vector store**.

A vector store is a database of embeddings that specializes in fast similarity searches. 

<img src='images/system4.png' width="1200">

To understand how a vector store works, let's create one in memory and add a few embeddings to it:

In [45]:
!pip install langchain[docarray]

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hnswlib
ERROR: Could not build wheels for hnswlib, which is required to install pyproject.toml-based projects



  Using cached docarray-0.32.1-py3-none-any.whl.metadata (30 kB)
  Using cached types_requests-2.31.0.20240406-py3-none-any.whl.metadata (1.8 kB)
  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached docarray-0.32.1-py3-none-any.whl (215 kB)
Using cached types_requests-2.31.0.20240406-py3-none-any.whl (15 kB)
Failed to build hnswlib


In [42]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

We can now query the vector store to find the most similar embeddings to a given query:

In [39]:
vectorstore1.similarity_search_with_score(query="Who is Mary's sister?", k=3)

[(Document(page_content="Mary's sister is Susana"), 0.9172681550033158),
 (Document(page_content='Mary has two siblings'), 0.9045628481161789),
 (Document(page_content='John and Tommy are brothers'), 0.8015500435454899)]

## Connecting the vector store to the chain

We can use the vector store to find the most relevant chunks from the transcription to send to the model. Here is how we can connect the vector store to the chain:

<img src='images/chain4.png' width="1200">

We need to configure a [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/). The retriever will run a similarity search in the vector store and return the most similar documents back to the next step in the chain.

We can get a retriever directly from the vector store we created before: 

In [40]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(page_content="Mary's sister is Susana"),
 Document(page_content='Mary has two siblings'),
 Document(page_content='John and Tommy are brothers'),
 Document(page_content="Pedro's mother is a teacher")]

Our prompt expects two parameters, "context" and "question." We can use the retriever to find the chunks we'll use as the context to answer the question.

We can create a map with the two inputs by using the [`RunnableParallel`](https://python.langchain.com/docs/expression_language/how_to/map) and [`RunnablePassthrough`](https://python.langchain.com/docs/expression_language/how_to/passthrough) classes. This will allow us to pass the context and question to the prompt as a map with the keys "context" and "question."

In [41]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What color is Patricia's car?")

{'context': [Document(page_content='Patricia likes white cars'),
  Document(page_content='Lucia drives an Audi'),
  Document(page_content="Pedro's mother is a teacher"),
  Document(page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

Let's now add the setup map to the chain and run it:



In [42]:
chain = setup | prompt | model | parser
chain.invoke("What color is Patricia's car?")

'White'

Let's invoke the chain using another example:

In [43]:
chain.invoke("What car does Lucia drive?")

'Lucia drives an Audi.'

## Loading transcription into the vector store

We initialized the vector store with a few random strings. Let's create a new vector store using the chunks from the video transcription.

In [44]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

Let's set up a new chain using the correct vector store. This time we are using a different equivalent syntax to specify the [`RunnableParallel`](https://python.langchain.com/docs/expression_language/how_to/map) portion of the chain:

In [45]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence is described as the next stage of development in the context provided. It is mentioned that synthetic intelligences will uncover puzzles in the universe and solve them.'

## Setting up Pinecone

So far we've used an in-memory vector store. In practice, we need a vector store that can handle large amounts of data and perform similarity searches at scale. For this example, we'll use [Pinecone](https://www.pinecone.io/).

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable `PINECONE_API_KEY`.

Then, we can load the transcription documents into Pinecone:

In [46]:
from langchain_pinecone import PineconeVectorStore

index_name = "youtube-rag-index"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

/Users/svpino/dev/youtube-rag/.venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Let's now run a similarity search on pinecone to make sure everything works:

In [47]:
pinecone.similarity_search("What is Hollywood going to start doing?")[:3]

[Document(page_content="It's like high quality audio and you're speaking usually pretty clearly. I don't know what open AI's plans are either. Yeah, there's always fun projects basically. And stable diffusion also is opening up a huge amount of experimentation. I would say in the visual realm and generating images and videos and movies. I'll think like videos now. And so that's going to be pretty crazy. That's going to almost certainly work and it's going to be really interesting when the cost of content creation is going to fall to zero. You used to need a painter for a few months to paint a thing and now it's going to be speak to your phone to get your video. So Hollywood will start using it to generate scenes, which completely opens up. Yeah, so you can make a movie like Avatar eventually for under a million dollars. Much less. Maybe just by talking to your phone. I mean, I know it sounds kind of crazy. And then there'd be some voting mechanism. Like how do you have a, like, would t

Let's setup the new chain using Pinecone as the vector store:

In [48]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("What is Hollywood going to start doing?")

'Hollywood is going to start using AI to generate scenes for movies.'